<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#import-packages" data-toc-modified-id="import-packages-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>import packages</a></span></li><li><span><a href="#env" data-toc-modified-id="env-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>env</a></span><ul class="toc-item"><li><span><a href="#c16.2" data-toc-modified-id="c16.2-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>c16.2</a></span></li></ul></li><li><span><a href="#agent" data-toc-modified-id="agent-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>agent</a></span></li><li><span><a href="#train" data-toc-modified-id="train-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>train</a></span></li></ul></div>

# Set up

## install packages

In [1]:
from google.colab import drive # import drive from google colab
drive.mount("/content/drive") 
!pip install neo
!pip install matplotlib_scalebar
!pip install ffmpeg
!pip install stable_baselines3
!pip install optuna

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 50.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.8/87.8 KB 8.3 MB/s eta 0:00:00
  Created wheel for neo: filename=neo-0.11.1-py3-none-any.whl size=579963 sha256=b87a5b526d20c8c6f05cd54d1ad3ea975a8cc8c56bfe72fdb4da815354fc3911
  Stored in directory: /root/.cache/pip/wheels/6a/63/0c/ec05bf584fd57600854a6832201659728466464abd19afebb4
Successfully built neo
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6084 sha256=8e5a3eeb465cf5206693700588d25614673bb49825ab8d8d9cf1ac5afd499997
  Stored in d

## import packages

In [2]:
%cd /content/drive/MyDrive/ff_repo/Multifirefly-Project
from model_storing_path import *
from config import *
from RL.env import *
from functions.SB3_functions import *
from functions.interpret_neural_network import interpret_neural_network_func
from functions.find_patterns import *
from functions.collect_agent_data import collect_agent_data_func
from functions.animation_func import *

import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import optuna
import matplotlib
import matplotlib.animation as animation
from matplotlib import rc, cm
from stable_baselines3 import SAC
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common import results_plotter
from stable_baselines3.common.results_plotter import plot_results
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from IPython.display import HTML
from functools import partial

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
torch.set_printoptions(sci_mode=False)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)

/content/drive/MyDrive/ff_repo/Multifirefly-Project


# Env

## regular

In [3]:
env = MultiFF()
env = Monitor(env, log_dir)

## one-ff

In [4]:
env = MultiFF(num_obs_ff=1)
env = Monitor(env, log_dir)

# Agent

## make agent

In [5]:
log_dir = "RL/SB3_stored_models/SB3_Feb_12"
os.makedirs(log_dir, exist_ok=True)


env = MultiFF()
env = Monitor(env, log_dir)


# For direct training
sac_model = SAC("MlpPolicy", 
            env,
            gamma=0.995,
            learning_rate=0.0015,
            batch_size=1024,
            target_update_interval=50,
            buffer_size=1000000,
            learning_starts=10000,
            train_freq=10,
            ent_coef=0.00083,
            policy_kwargs=dict(activation_fn=nn.Tanh, net_arch=[128, 128])
                )

## load agent (optional)

In [6]:
path = os.path.join(log_dir, 'best_model.zip')
path2 = os.path.join(log_dir, 'buffer.pkl')
sac_model = sac_model.load(path,env=env) 
sac_model.load_replay_buffer(path2)

FileNotFoundError: ignored

# Train

In [7]:
callback = SaveOnBestTrainingRewardCallback(check_freq=20000, log_dir=log_dir)
timesteps = 50000000
sac_model.learn(total_timesteps=int(timesteps), callback=callback)
plot_results([log_dir], timesteps, results_plotter.X_TIMESTEPS, "Multiff")
plt.show()

/content/drive/MyDrive/ff_repo/Multifirefly-Project/RL/env.py:629: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ff_flashing_durations = torch.tensor(ff_flash[index])


188.0 sys_vel:  [0.1673, 0.0062] n_targets:  1
Reward for the episode:  100

 episode:  1
196.25 sys_vel:  [0.2583, 0.0014] n_targets:  1
Reward for the episode:  100

 episode:  2
Reward for the episode:  0

 episode:  3
57.25 sys_vel:  [0.9064, 0.0008] n_targets:  2
Reward for the episode:  200

 episode:  4
Reward for the episode:  0

 episode:  5
Reward for the episode:  0

 episode:  6
115.0 sys_vel:  [-0.2915, 0.0069] n_targets:  1
Reward for the episode:  100

 episode:  7
Reward for the episode:  0

 episode:  8
Reward for the episode:  0

 episode:  9
140.0 sys_vel:  [0.1292, 0.0088] n_targets:  1
213.25 sys_vel:  [0.2117, 0.0042] n_targets:  1
Reward for the episode:  200

 episode:  10
55.0 sys_vel:  [0.7598, 0.0043] n_targets:  1
Reward for the episode:  100

 episode:  11
82.5 sys_vel:  [0.4219, 0.0044] n_targets:  1
117.0 sys_vel:  [0.9291, 0.0094] n_targets:  1
134.75 sys_vel:  [-0.8049, 0.0042] n_targets:  1
Reward for the episode:  300

 episode:  12
120.5 sys_vel:  [0

KeyboardInterrupt: ignored

# Test

In [ ]:
obs = env.reset()
cum_rewards = 0
for step in range(1000):
    action, _ = sac_model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    cum_rewards += reward
    if done:
        obs = env.reset()
    # print(step, ffxy_visible[-1])
print(cum_rewards)

# Hyperparameter tuning

## parameters to sample from

In [ ]:
def sample_sac_params(trial):
    """
    Sampler for SAC hyperparams.


    Parameters
    ----------
    trial: (optuna.trial)

    Return: (dict)
    """


    gamma = 1.0 - trial.suggest_float("gamma", 0.0001, 0.1, log=True)
    learning_rate = trial.suggest_float("lr", 1e-5, 1, log=True)
    tau = trial.suggest_float("tau", 1e-6, 1, log=True)
    #batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128, 256, 512, 1024])
    target_update_interval = trial.suggest_categorical('target_update_interval', [5, 10, 20, 40, 60, 100, 200])
    #buffer_size = trial.suggest_categorical('buffer_size', [int(1e5), int(1e6)]) # This actually doesn't matter much here because of limited timesteps
    learning_starts = trial.suggest_categorical('learning_starts', [5000, 10000, 15000])
    train_freq = trial.suggest_categorical('train_freq', [1, 10, 100, 300])
    ## gradient_steps takes too much time
    # gradient_steps = trial.suggest_categorical('gradient_steps', [1, 100, 300])
    gradient_steps = train_freq
    ent_coef = trial.suggest_float("ent_coef", 0.00000001, 0.1, log=True)
    net_arch = trial.suggest_categorical('net_arch', ["small", "medium", "big"])
    activation_fn = trial.suggest_categorical("activation_fn", ["tanh"])

    net_arch = {
        'small': [100, 100],
        'medium': [128, 128],
        'big': [200, 200],
    }[net_arch]

    activation_fn = {"tanh": nn.Tanh, "relu": nn.ReLU}[activation_fn]

    target_entropy = 'auto'
    if ent_coef == 'auto':
        target_entropy = trial.suggest_categorical('target_entropy', ['auto', -1, -10, -20, -50, -100])


    ## Display true values
    # trial.set_user_attr("gamma_", gamma)
    # trial.set_user_attr("n_steps", n_steps)


    return {
        'gamma': gamma,
        'learning_rate': learning_rate,
        'tau': tau,
        #'batch_size': batch_size,
        'target_update_interval': target_update_interval,
        #'buffer_size': buffer_size,
        'learning_starts': learning_starts,
        'train_freq': train_freq,
        'gradient_steps': gradient_steps,
        'ent_coef': ent_coef,
        'target_entropy': target_entropy,
        'policy_kwargs': {
            "net_arch": net_arch,
            "activation_fn": activation_fn
        }
    }

## objective

In [ ]:
def objective(trial: optuna.Trial) -> float:
    kwargs = DEFAULT_HYPERPARAMS.copy()
    # Sample hyperparameters
    kwargs.update(sample_sac_params(trial))
    # Create the RL model
    model = SAC(**kwargs)
    # Create env used for evaluation
    eval_env = env
    # Create the callback that will periodically evaluate
    # and report the performance
    eval_callback = TrialEvalCallback(
      eval_env, trial, n_eval_episodes=N_EVAL_EPISODES, eval_freq=EVAL_FREQ, deterministic=True
    )

    nan_encountered = False
    try:
      model.learn(N_TIMESTEPS, callback=eval_callback)
    except AssertionError as e:
      # Sometimes, random hyperparams can generate NaN
      print(e)
      nan_encountered = True
    finally:
      # Free memory
      model.env.close()
      eval_env.close()

    # Tell the optimizer that the trial failed
    if nan_encountered:
      return float("nan")

    if eval_callback.is_pruned:
      raise optuna.exceptions.TrialPruned()

    return eval_callback.last_mean_reward

## run

In [ ]:
env = MultiFF()
env.reset()



DEFAULT_HYPERPARAMS = {
    "policy": "MlpPolicy",
    "env": env,
}

N_TRIALS = 100
N_STARTUP_TRIALS = 5
N_EVALUATIONS = 2


N_TIMESTEPS = 100000
EVAL_FREQ = int(N_TIMESTEPS / N_EVALUATIONS)
N_EVAL_EPISODES = 1



# Set pytorch num threads to 1 for faster training
torch.set_num_threads(1)

sampler = TPESampler(n_startup_trials=N_STARTUP_TRIALS)
# Do not prune before 1/3 of the max budget is used
pruner = MedianPruner(n_startup_trials=N_STARTUP_TRIALS, n_warmup_steps=N_EVALUATIONS//3)

study = optuna.create_study(sampler=sampler, pruner=pruner, direction="maximize")
try:
    study.optimize(objective, n_trials=N_TRIALS)
except KeyboardInterrupt:
    pass

print("Number of finished trials: ", len(study.trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

print("  User attrs:")
for key, value in trial.user_attrs.items():
    print("    {}: {}".format(key, value))


# Animation

## collect data

In [ ]:
env = CollectInformation()

monkey_information, ff_flash_sorted, ff_catched_T_sorted, ff_believed_position_sorted, \
           ff_real_position_sorted, ff_life_sorted, ff_flash_end_sorted, catched_ff_num, total_ff_num, \
           obs_ff_unique_identifiers, sorted_indices_all \
           = collect_agent_data_func(env, sac_model, n_steps = 2000, LSTM = False)


ff_dataframe = make_ff_dataframe(monkey_information, ff_catched_T_sorted, ff_flash_sorted,  ff_real_position_sorted, ff_life_sorted, \
                                 player = "agent", max_distance = 400, data_folder_name = None, num_missed_index = 0, truncate = False, \
                                 obs_ff_unique_identifiers = obs_ff_unique_identifiers, sorted_indices_all = sorted_indices_all)

min_point_index, max_point_index = np.min(np.array(ff_dataframe['point_index'])), np.max(np.array(ff_dataframe['point_index']))

n_ff_in_a_row = n_ff_in_a_row_func(ff_believed_position_sorted, distance_between_ff = 50)
visible_before_last_one_trials = visible_before_last_one_func(ff_dataframe)
disappear_latest_trials = disappear_latest_func(ff_dataframe)
ignore_sudden_flash_trials, ignore_sudden_flash_indices, ignore_sudden_flash_indices_for_anim = ignore_sudden_flash_func(ff_dataframe, ff_real_position_sorted, max_point_index)
try_a_few_times_trials, try_a_few_times_indices, try_a_few_times_indices_for_anim = try_a_few_times_func(ff_catched_T_sorted, monkey_information, ff_believed_position_sorted, PLAYER, max_point_index)
give_up_after_trying_trials, give_up_after_trying_indices, give_up_after_trying_indices_for_anim = give_up_after_trying_func(ff_catched_T_sorted, monkey_information, ff_believed_position_sorted, PLAYER, max_point_index)

annotation_info = make_annotation_info(catched_ff_num, max_point_index, n_ff_in_a_row, visible_before_last_one_trials, disappear_latest_trials, \
                                       ignore_sudden_flash_indices, give_up_after_trying_indices, try_a_few_times_indices)

## prepare for animation

In [ ]:
currentTrial = 2
num_trials = 7
margin = 400
k = 5

duration = [ff_catched_T_sorted[currentTrial-1], ff_catched_T_sorted[currentTrial+num_trials-1]]
cum_index = np.where((monkey_information['monkey_t'] > ff_catched_T_sorted[currentTrial-1]) & 
                     (monkey_information['monkey_t'] <= ff_catched_T_sorted[currentTrial+num_trials-1]))
if len(cum_index) > 0:
    anim_monkey_info = make_anim_monkey_info(monkey_information, cum_index, k = k)
    flash_on_ff_dict = flash_on_ff_in_trial_by_time(anim_monkey_info['anim_t'], currentTrial, num_trials, ff_flash_sorted, ff_life_sorted, ff_catched_T_sorted)
    believed_ff_dict = believed_ff(anim_monkey_info['anim_t'], currentTrial, num_trials, ff_believed_position_sorted, ff_catched_T_sorted)

    num_frames = anim_monkey_info['anim_t'].size
    ff_position_during_this_trial = np.array([ff_real_position_sorted[ff_index] for ff_index, life in \
                                    enumerate(ff_life_sorted) if (life[-1] >= duration[0]) and (life[0] < duration[1])])
else:
    print("Please try another number for currentTrial, or increase num_trials.")

plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['font.size'] = 15
fig, ax = plt.subplots()

## make animation

In [ ]:
animate_func = partial(animate, ax=ax, anim_monkey_info=anim_monkey_info, ff_dataframe=ff_dataframe, ff_real_position_sorted=ff_real_position_sorted, ff_position_during_this_trial=ff_position_during_this_trial, \
                         flash_on_ff_dict=flash_on_ff_dict, believed_ff_dict=believed_ff_dict, margin = 400)
anim = animation.FuncAnimation(fig, animate_func, frames=num_frames, interval=100, repeat=True) 
HTML(anim.to_html5_video())

## make animation with annotation

In [ ]:
animate_annotated_func = partial(animate_annotated, ax=ax, anim_monkey_info=anim_monkey_info, margin=margin, ff_dataframe=ff_dataframe, ff_real_position_sorted=ff_real_position_sorted, ff_position_during_this_trial=ff_position_during_this_trial, \
                                   flash_on_ff_dict=flash_on_ff_dict, believed_ff_dict=believed_ff_dict, ff_catched_T_sorted=ff_catched_T_sorted, annotation_info=annotation_info)
anim_annotated = animation.FuncAnimation(fig, animate_annotated_func, frames=num_frames, interval=100, repeat=True) 
HTML(anim_annotated.to_html5_video())

# Interpret neural network

## angle vs. distance, dv

In [ ]:
interpret_neural_network_func( sac_model, sample_size = 1000,const_memory = 4, plot_in_xy_coord=True)

## angle vs. distance, dw

In [ ]:
interpret_neural_network_func( sac_model, sample_size = 1000,const_memory = 4, color_variable="dw", plot_in_xy_coord=True)

## angle vs. memory, dv

In [ ]:
interpret_neural_network_func( sac_model, sample_size = 1000, const_distance = 100, color_variable="dv", plot_in_xy_coord=True)

## angle vs. memory, dw

In [ ]:
interpret_neural_network_func( sac_model, sample_size = 1000, const_distance = 100, color_variable="dw", plot_in_xy_coord=True)

## distance vs. memory, dv

In [ ]:
interpret_neural_network_func( sac_model, sample_size = 1000, const_angle = 0.5, color_variable="dv", plot_in_xy_coord=True)

## distance vs. memory, dw

In [ ]:
interpret_neural_network_func( sac_model, sample_size = 1000, const_angle = 0.5, color_variable="dw", plot_in_xy_coord=True)